<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install yfinance==0.2.59

  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.31.1
    Uninstalling protobuf-6.31.1:
      Successfully uninstalled protobuf-6.31.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.65
    Uninstalling yfinance-0.2.65:
      Successfully uninstalled yfinance-0.2.65
   ━━━━━━━━━━━━━━━━━━━━╺━━━━━━━━━━━━━━━━━━━ 1/2 [yfinance]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-07-08 09:15:31
-------------------
qualified stocks: 96
with latest results: 31
still star stocks: 20
-------------------
Initial Investment:  1.17 C
CY Investment:  1.44 C
Reserve:  9.00 K
Current:  1.37 C
-------------------
Today PnL: 44.86 K (0.33%)
Current PnL: -12.98 L (-9.04%)
CY Booked + Current PnL: -6.02 L (-4.19%)
-------------------
Total profit:  4.81 L
Total loss:  -17.79 L
-------------------
Total Booked + Current PnL: 20.96 L (17.99%)
Total Booked PnL: 33.94 L (29.12%)
Curr Year Booked PnL: 6.96 L (5.07%)
Prev Year Booked PnL: 26.98 L (23.15%)
Est FTT:  2.09 C
Est FTT PnL: 71.44 L (52.09%)
Deployed:  1.17 C
Current:  1.37 C
CAGR/XIRR %: 10.98%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'AvgCost', 'FTT', 'Dev%_PE', 'Conviction', 'RSP', 'Current', 'Current P/L', 'FTT Amt', 'Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
4,APOLLOHOSP,6904.43,8285.00,-16.19,H-LC,92.71,114195.0,10629.0,10083.0,-0.04,10.26,8.83,20.00,32.0,1.05,0.83,25.78,X40N,BTT,HEALTHCARE
39,HINDALCO,651.95,761.55,-19.90,H-LC,84.38,110568.0,6256.0,11278.0,-0.15,6.00,10.20,16.81,27.0,0.55,0.81,23.01,X5K,ATH,METALS
88,UNIONBANK,123.87,163.00,0.15,M-LC,27.08,173586.0,32746.0,11752.0,0.33,23.25,6.77,31.59,88.0,2.79,1.27,50.79,XY24,NTT,BANKS
92,VIPIND,488.80,489.00,-212.52,H-SC,97.92,81188.0,-14128.0,14167.0,0.37,-14.82,17.45,0.04,157.0,-1.00,0.59,60.13,OX40N,NTT,MISC
71,SBIN,760.30,863.00,-11.51,M-LC,30.21,214054.0,12574.0,14641.0,0.11,6.24,6.84,13.51,77.0,0.86,1.56,19.62,XY25,NTT,BANKS


In [10]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
12,BANKINDIA,116.91,190.00,-31.29,M-MC,11.46,180961.0,1153.0,111255.0,-0.05,0.64,61.48,62.52,173.0,0.01,1.32,34.95,XR,NTT,BANKS
38,HEROMOTOCO,4311.81,6039.03,-6.19,H-MC,33.33,151193.0,280.0,60175.0,0.30,0.19,39.80,40.06,103.0,0.00,1.10,23.67,AR,ATH,AUTO
60,NATIONALUM,189.63,247.44,-50.15,H-MC,53.12,100340.0,-164.0,30804.0,0.40,-0.16,30.70,30.49,97.0,-0.01,0.73,32.19,MH,ATH,METALS


In [11]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.2,-14.84,L-SC,100.00,171528.0,29686.0,26878.0,19.52,20.93,15.67,39.88,262.0,1.10,1.25,73.83,XY24,NTT,POWER
0,5PAISA,564.75,565.0,164.42,H-SC,43.75,142644.0,-44288.0,44377.0,1.76,-23.69,31.11,0.04,149.0,-1.00,1.04,36.46,OX40N,NTT,FINANCE
85,TRIDENT,37.35,48.0,23.89,M-SC,65.10,77805.0,-14450.0,40754.0,1.22,-15.66,52.38,28.51,233.0,-0.35,0.57,33.14,XR,NTT,TEXTILES
32,ENRIN,1377.95,3034.1,NaN,NaN,93.75,75852.0,41403.0,0.0,1.13,120.19,0.00,120.19,274.0,inf,0.55,15.99,AR,ATH,ELECTRICAL
72,SFL,1016.35,1287.0,35.26,H-SC,86.46,178652.0,-83566.0,153391.0,0.91,-31.87,85.86,26.63,150.0,-0.54,1.30,11.08,XY24,NTT,MISC


In [12]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
90,VALIANTORG,512.64,838.00,-352.03,H-SC,5.21,107146.0,-26140.0,110735.0,-1.20,-19.61,103.35,63.47,151.0,-0.24,0.78,79.60,XR,NTT,CHEMICALS
16,BLUSPRING,226.45,84.55,NaN,NaN,15.62,17682.0,-31911.0,835.0,-1.14,-64.35,4.72,-62.66,270.0,-38.22,0.13,18.58,XY24,ATH,MISC
63,QUESS,296.83,986.00,-33.33,M-SC,19.79,66248.0,1242.0,149687.0,-0.97,1.91,225.95,232.18,193.0,0.01,0.48,1.44,XY24,NTT,MISC
15,BLUESTARCO,1646.70,2337.55,-3.43,H-SC,96.88,181040.0,16370.0,52719.0,-0.89,9.94,29.12,41.95,124.0,0.31,1.32,18.15,X40N,ATH,AC
34,HAPPSTMNDS,919.80,1496.95,-3.05,M-SC,78.12,67610.0,-28969.0,89570.0,-0.52,-30.00,132.48,62.75,214.0,-0.32,0.49,18.55,AR,ATH,IT


In [13]:
# OX40N stocks
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
87,UJJIVANSFB,52.77,53.0,47.02,M-SC,23.96,127494.0,-14985.0,15605.0,0.47,-10.52,12.24,0.44,247.0,-0.96,0.93,50.48,OX40N,NTT,BANKS
79,SYMPHONY,1306.42,1306.0,-28.99,M-SC,29.17,151357.0,-19784.0,19722.0,0.34,-11.56,13.03,-0.03,191.0,-1.00,1.10,8.78,OX40N,NTT,DURABLES
46,INDIGOPNTS,1407.73,1408.0,127.68,H-SC,94.79,149122.0,-25437.0,25470.0,0.15,-14.57,17.08,0.02,133.0,-1.00,1.09,29.31,OX40N,NTT,PAINTS
92,VIPIND,488.80,489.0,-212.52,H-SC,97.92,81188.0,-14128.0,14167.0,0.37,-14.82,17.45,0.04,157.0,-1.00,0.59,60.13,OX40N,NTT,MISC
81,TATAELXSI,7332.28,7332.0,-17.48,X-MC,10.42,86646.0,-16006.0,16004.0,0.23,-15.59,18.47,-0.00,59.0,-1.00,0.63,31.71,OX40N,NTT,IT


In [14]:
# top 5 to accumulate
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
83,TCS,3794.03,4998.00,-18.38,X-LC,37.50,265699.0,-30235.0,124135.0,-0.16,-10.22,46.72,31.73,1.0,-0.24,1.94,6.35,X40,BTT,IT
48,INFY,1461.46,2275.00,-9.19,X-LC,71.88,291036.0,29435.0,116182.0,-0.07,11.25,39.92,55.67,2.0,0.25,2.12,18.04,X40,BTT,IT
40,HINDUNILVR,2413.81,2723.00,-13.95,X-LC,59.38,269618.0,-729.0,35347.0,-0.13,-0.27,13.11,12.81,5.0,-0.02,1.97,12.76,XY25,NTT,FMCG
17,BRITANNIA,4983.33,6446.05,18.31,X-LC,72.92,271377.0,42144.0,25130.0,0.26,18.38,9.26,29.35,10.0,1.68,1.98,28.95,XY25,ATH,FMCG
74,SIEMENS,4406.90,7969.85,-2.42,X-LC,26.04,82420.0,-27752.0,116830.0,0.57,-25.19,141.75,80.85,11.0,-0.24,0.60,21.39,AR,ATH,ELECTRICAL


In [15]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3034.10,NaN,NaN,93.75,75852.0,41403.0,0.0,1.13,120.19,0.00,120.19,274.0,inf,0.55,15.99,AR,ATH,ELECTRICAL
28,DIGITIDE,188.38,237.72,NaN,NaN,31.25,50260.0,9005.0,1799.0,0.44,21.83,3.58,26.19,272.0,5.01,0.37,18.33,XY24,ATH,IT
52,JPPOWER,18.73,26.20,-14.84,L-SC,100.00,171528.0,29686.0,26878.0,19.52,20.93,15.67,39.88,262.0,1.10,1.25,73.83,XY24,NTT,POWER
88,UNIONBANK,123.87,163.00,0.15,M-LC,27.08,173586.0,32746.0,11752.0,0.33,23.25,6.77,31.59,88.0,2.79,1.27,50.79,XY24,NTT,BANKS


In [16]:
# top 5 to monitor
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
68,SAIL,130.64,228.00,91.93,M-MC,38.54,230180.0,5218.0,162438.0,0.15,2.32,70.57,74.53,189.0,0.03,1.68,33.64,XY24,BTT,STEEL
54,KPIGREEN,497.21,731.82,25.90,H-SC,39.58,128671.0,3374.0,55753.0,-0.46,2.69,43.33,47.19,143.0,0.06,0.94,60.98,MH,ATH,POWER
20,CAMS,3643.00,5250.99,3.43,H-SC,22.92,116054.0,14050.0,30975.0,0.07,13.77,26.69,44.14,126.0,0.45,0.85,32.25,X40N,ATH,MISC
88,UNIONBANK,123.87,163.00,0.15,M-LC,27.08,173586.0,32746.0,11752.0,0.33,23.25,6.77,31.59,88.0,2.79,1.27,50.79,XY24,NTT,BANKS
95,WIPRO,243.46,420.00,-2.86,M-LC,87.50,166439.0,15494.0,93955.0,0.26,10.26,56.45,72.51,71.0,0.16,1.21,14.65,XR,NTT,IT


In [17]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
32,ENRIN,1377.95,3034.10,NaN,NaN,93.75,75852.0,41403.0,0.0,1.13,120.19,0.00,120.19,274.0,inf,0.55,15.99,AR,ATH,ELECTRICAL
94,WHIRLPOOL,1167.49,2270.00,-38.56,M-SC,58.33,202757.0,31136.0,130940.0,0.47,18.14,64.58,94.43,201.0,0.24,1.48,49.65,XR,NTT,DURABLES
95,WIPRO,243.46,420.00,-2.86,M-LC,87.50,166439.0,15494.0,93955.0,0.26,10.26,56.45,72.51,71.0,0.16,1.21,14.65,XR,NTT,IT
45,INDIAMART,2327.09,4850.92,-48.08,H-SC,68.75,135028.0,11692.0,122065.0,0.37,9.48,90.40,108.45,123.0,0.10,0.99,33.95,AR,ATH,MISC
54,KPIGREEN,497.21,731.82,25.90,H-SC,39.58,128671.0,3374.0,55753.0,-0.46,2.69,43.33,47.19,143.0,0.06,0.94,60.98,MH,ATH,POWER


In [18]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
16,BLUSPRING,226.45,84.55,NaN,NaN,15.62,17682.0,-31911.0,835.0,-1.14,-64.35,4.72,-62.66,270.0,-38.22,0.13,18.58,XY24,ATH,MISC
24,COFFEEDAY,59.14,80.00,-55.73,L-SC,76.04,65818.0,-47731.0,87781.0,0.59,-42.04,133.37,35.27,265.0,-0.54,0.48,61.09,XR,NTT,HOTELS
32,ENRIN,1377.95,3034.10,NaN,NaN,93.75,75852.0,41403.0,0.0,1.13,120.19,0.00,120.19,274.0,inf,0.55,15.99,AR,ATH,ELECTRICAL
58,MASFIN,326.60,399.50,-18.25,H-SC,67.71,92850.0,-5130.0,27001.0,0.11,-5.24,29.08,22.32,145.0,-0.19,0.68,34.66,XR,ATH,FINANCE
9,BAJAJFINSV,1994.80,2758.00,-1.80,X-LC,36.46,98093.0,348.0,37050.0,-0.22,0.36,37.77,38.26,18.0,0.01,0.72,30.98,X40,BTT,FINANCE


In [19]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,AvgCost,FTT,Dev%_PE,Conviction,RSP,Current,Current P/L,FTT Amt,Today P/L%,Current P/L%,FTT%,OTT%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
52,JPPOWER,18.73,26.20,-14.84,L-SC,100.00,171528.0,29686.0,26878.0,19.52,20.93,15.67,39.88,262.0,1.10,1.25,73.83,XY24,NTT,POWER
3,ALKYLAMINE,2347.98,4546.37,10.46,X-SC,98.96,102839.0,1876.0,92658.0,-0.27,1.86,90.10,93.63,67.0,0.02,0.75,53.18,SR,ATH,CHEMICALS
92,VIPIND,488.80,489.00,-212.52,H-SC,97.92,81188.0,-14128.0,14167.0,0.37,-14.82,17.45,0.04,157.0,-1.00,0.59,60.13,OX40N,NTT,MISC
15,BLUESTARCO,1646.70,2337.55,-3.43,H-SC,96.88,181040.0,16370.0,52719.0,-0.89,9.94,29.12,41.95,124.0,0.31,1.32,18.15,X40N,ATH,AC
65,RELAXO,902.64,1176.00,-28.44,H-SC,95.83,64698.0,-50840.0,85828.0,-0.18,-44.00,132.66,30.28,131.0,-0.59,0.47,26.39,X40N,NTT,FOOTWEAR


In [20]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,18.92
1,25,41.90
2,50,70.59


In [21]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    43.52
LC    33.95
MC    21.51
Name: CurrAlloc%, dtype: float64

In [22]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     30.90
X40      12.91
XY25     12.06
X40N     11.95
XR       10.03
OX40N     8.21
AR        8.20
MH        1.67
X5K       1.45
X200      1.41
SR        1.24
Name: CurrAlloc%, dtype: float64

In [23]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    20.51
M-SC    15.91
H-LC    14.11
X-LC    12.92
H-MC     9.50
X-MC     7.61
M-LC     5.88
X-SC     4.44
M-MC     4.04
L-SC     2.66
L-LC     1.04
L-MC     0.36
Name: CurrAlloc%, dtype: float64

In [24]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,15.24,-0.23,33.81
IT,12.09,-9.65,63.50
FINANCE,8.53,-11.51,51.98
BANKS,7.74,-7.58,51.47
MISC,5.83,-20.22,76.17
PAINTS,5.68,-12.08,34.17
ELECTRICAL,5.47,1.56,44.60
HEALTHCARE,4.62,-1.77,31.50
AUTO,4.33,-14.07,64.22


In [25]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,2654129.0,26
XR,1084799.0,13
AR,1022496.0,10
X40,686896.0,10
X40N,527022.0,11
XY25,467063.0,8
OX40N,303485.0,11
SR,174445.0,2
MH,86557.0,2


In [26]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,1973754.0,20
M-SC,1762289.0,21
H-LC,606937.0,12
X-LC,540114.0,8
H-MC,530454.0,8
M-MC,415757.0,3
X-MC,412030.0,6
X-SC,310726.0,5
M-LC,255385.0,4


In [27]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                 
H-SC       XY24      884390.0      6
M-SC       XY24      806930.0      7
H-SC       AR        523791.0      3
M-SC       XR        334617.0      4
M-MC       XY24      304502.0      2
H-MC       XY24      300722.0      3
X-LC       X40       277367.0      3
H-SC       XR        218282.0      3
           X40N      207524.0      4
H-LC       X40N      192636.0      4
M-SC       AR        183518.0      2
           OX40N     162851.0      5
H-LC       X40       155390.0      3
X-MC       X40       154662.0      2
L-SC       XR        149298.0      2
M-LC       XY24      146789.0      2
X-SC       XR        123462.0      1
M-SC       XY25      121096.0      1
X-LC       AR        116830.0      1
M-MC       XR        111255.0      1
X-MC       XY25      101950.0      1
X-LC       XY25       99414.0      3
M-LC       XR         93955.0      1
X-SC       SR         92658.0      1
X-MC       XY24       87046.0      1
H-SC       OX40N      84014.0      3
H-MC       XY25       82775.0      1
M-SC       SR         81787.0      1
H-LC       AR         81219.0      1
M-SC       X40        71490.0      1
H-LC       X5K        69835.0      2
           X200       67609.0      1
H-MC       AR         60175.0      1
L-SC       AR         56963.0      1
H-SC       MH         55753.0      1
X-SC       XY24       53990.0      1
L-MC       XR         53930.0      1
X-MC       X40N       52368.0      1
L-LC       XY25       47187.0      1
X-LC       X40N       46503.0      1
X-SC       OX40N      40616.0      2
H-LC       XY24       40248.0      1
H-MC       MH         30804.0      1
           X40N       27991.0      1
           X40        27987.0      1
L-SC       XY24       26878.0      1
X-MC       OX40N      16004.0      1
M-LC       XY25       14641.0      1

In [28]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 27.0 seconds
